# VAE for text generation

Just a little experiment I'm running to see if I can do it. I have an agentic version of this repo too, but want to make sure I can code it from scratch as well.

My goal is to get a network that can generate synthetic tweets after being trained on an appropriately sized dataset... or at least could, in principle. I'll need to learn more about how to set up recurrent architectures than I currently know, so this is a good exercise for me!

In [ ]:
# imports
import torch
import torch.nn as nn

In [ ]:
class TextVAE(nn.Module):
    # A VAE that recreates text. Specifically, I'll use it for tweets.

    def __init__(
        self,
        vocab_size,
        embedding_dim,
        hidden_dim,        
        num_layers,
        dropout,
        latent_dim
    ):
        """
        Initialize the VAE.

        Inputs:
        ----
        vocab_size: Size of the model vocabulary. Recommend to set equal 
                    to the number of unique tokens in the training data.
        embedding_dim: Dimension of the space in which tokens are embedded.
        hidden_dim: Dimension of the hidden layer between the GRU and the
                    latent space.
        num_layers: Number of layers in each GRU.
        dropout: Dropout probability for the GRU.
        latent_dim: Dimension of the latent space
        """
        super().__init__()

        # Encoder
        # ----

        # Encoder embedding layer: Convert tokens into vectors. 
        # Input: tensor (batch_size, seq_length)
        # Output: tensor (batch_size, seq_length, embedding_dim)
        self.encoder_embedding = nn.Embedding(
            vocab_size,
            embedding_dim,
            padding_idx=0,
        )

        # Encoder GRU: Reads tokens into GRU, a recurrent module which can handle the
        # sequence of tokens. Since we're using the hidden states, be mindful to transpose
        # things so that the tensor has the right shape afterwards!
        # Input: tensor (batch_size, seq_length, embedding_dim)
        # Output: output tensor (batch_size, seq_length, hidden_dim)
        #         hidden state tensor (num_layers, batch_size, hidden_dim)
        self.encoder_gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
            bidirectional=False,
        )

        # Latent space
        # ----

        # Fully connected layers: One each for mu and log variance. In both cases:
        # Input: tensor (batch_size, seq_length, hidden_dim)
        # Output: tensor (batch_size, seq_length, latent_dim)
        self.fc_mu = nn.Linear(
            in_features=hidden_dim, 
            out_features=latent_dim,
        )
        self.fc_logvar = nn.Linear(
            in_features=hidden_dim, 
            out_features=latent_dim,
        )

        # Decoder
        # ----
        
        # Fully connected layer: Going from the latent space to the decoder's GRU.
        # Input: tensor (batch_size, seq_length, latent_dim)
        # Output: tensor (batch_size, seq_length, hidden_dim)
        self.fc_decoder_init = nn.Linear(
            in_features=latent_dim,
            out_features=hidden_dim
        )

        # Decoder embedding layer: Convert max hidden state indices (i.e. tokens) into vectors
        # Input: tensor (batch_size, seq_length)
        # Output: tensor (batch_size, seq_length, embedding_dim)
        self.decoder_embedding = nn.Embedding(
            vocab_size,
            embedding_dim,
            padding_idx=0,
        )

        # Decoder GRU: The recurrent part of the decoder. Just like the encoder!
        # Input: tensor (batch_size, seq_length, embedding_dim)
        # Output: output tensor (batch_size, seq_length, hidden_dim)
        #         hidden state tensor (num_layers, batch_size, hidden_dim)
        self.decoder_gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
            bidirectional=False,
        )

        # Output layer: Convert GRU hidden states into token logits.
        # Input: tensor (batch_size, seq_length, hidden_dim)
        # Output: tensor (batch_size, seq_length, vocab_size)
        self.fc_decoder_output = nn.Linear(
            in_features=hidden_dim,
            out_features=vocab_size
        )

